In [1]:


from nltk.tokenize import sent_tokenize

from docx import Document 
import re
import pandas as pd

from sentence_transformers import SentenceTransformer

import numpy as np

from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer

/Users/grealish/anaconda3/envs/interview/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# extract topics from interviewer tagged material
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [2]:
t = WordPunctTokenizer()
s = PorterStemmer()

In [3]:
sentence = "Provided math tutoring (algebra through trigonometry) to co-workers and their children, supporting colleagues' career advancement by preparing them for admissions tests for further education, including nursing programs."

In [4]:
t.tokenize(sentence)

['Provided',
 'math',
 'tutoring',
 '(',
 'algebra',
 'through',
 'trigonometry',
 ')',
 'to',
 'co',
 '-',
 'workers',
 'and',
 'their',
 'children',
 ',',
 'supporting',
 'colleagues',
 "'",
 'career',
 'advancement',
 'by',
 'preparing',
 'them',
 'for',
 'admissions',
 'tests',
 'for',
 'further',
 'education',
 ',',
 'including',
 'nursing',
 'programs',
 '.']

In [6]:
[s.stem(x) for x in t.tokenize(sentence)]

['provid',
 'math',
 'tutor',
 '(',
 'algebra',
 'through',
 'trigonometri',
 ')',
 'to',
 'co',
 '-',
 'worker',
 'and',
 'their',
 'children',
 ',',
 'support',
 'colleagu',
 "'",
 'career',
 'advanc',
 'by',
 'prepar',
 'them',
 'for',
 'admiss',
 'test',
 'for',
 'further',
 'educ',
 ',',
 'includ',
 'nurs',
 'program',
 '.']

In [10]:
test = pd.read_csv("df_tagged/Interview 01 Transcript tagged.csv")
test

,Speaker,Dialogue
0,Unknown,Interview Transcript
1,Unknown,Participant 1
2,Interviewer,"Okay, so I wanted to begin with just some basi..."
3,Participant,Okay I'm (Participant 1). Originally I'm from ...
4,Interviewer,"wow yeah. Wait, were you in the Olympics?"
...,...,...
145,Interviewer,"Um should I email you, I mean I guess I alread..."
146,Participant,"yeah, yeah. [name] send me a flyer so I will f..."
147,Interviewer,thank you so much that would be amazing
148,Participant,you’re welcome! Okay.


In [14]:
test_int = test[test['Speaker']=='Interviewer']
test_int

,Speaker,Dialogue
2,Interviewer,"Okay, so I wanted to begin with just some basi..."
4,Interviewer,"wow yeah. Wait, were you in the Olympics?"
6,Interviewer,"awesome yeah that's really, that's a good idea..."
8,Interviewer,"um so, can you tell me a little bit about your..."
10,Interviewer,gotcha
...,...,...
141,Interviewer,oh okay.
143,Interviewer,that would be amazing. Yeah that would be so a...
145,Interviewer,"Um should I email you, I mean I guess I alread..."
147,Interviewer,thank you so much that would be amazing


In [15]:
test_dia = test_int["Dialogue"]
test_dia

2      Okay, so I wanted to begin with just some basi...
4              wow yeah. Wait, were you in the Olympics?
6      awesome yeah that's really, that's a good idea...
8      um so, can you tell me a little bit about your...
10                                                gotcha
                             ...                        
141                                             oh okay.
143    that would be amazing. Yeah that would be so a...
145    Um should I email you, I mean I guess I alread...
147              thank you so much that would be amazing
149    Um oh real quick, uh do you have venmo or paypal?
Name: Dialogue, Length: 70, dtype: object

In [22]:
tokenized = []
stemmed = []
for sent in test_dia:
    tokenized.append(t.tokenize(sent))
    stemmed.append([s.stem(x) for x in t.tokenize(sent)])
    
tokenized
    

[['Okay',
  ',',
  'so',
  'I',
  'wanted',
  'to',
  'begin',
  'with',
  'just',
  'some',
  'basic',
  'demographic',
  'information',
  'background',
  '.',
  'Can',
  'you',
  'just',
  'give',
  'me',
  'a',
  'little',
  'summary',
  'of',
  'who',
  'you',
  'are',
  'and',
  'your',
  'education',
  'and',
  'employment',
  'history',
  '?'],
 ['wow',
  'yeah',
  '.',
  'Wait',
  ',',
  'were',
  'you',
  'in',
  'the',
  'Olympics',
  '?'],
 ['awesome',
  'yeah',
  'that',
  "'",
  's',
  'really',
  ',',
  'that',
  "'",
  's',
  'a',
  'good',
  'idea',
  'to',
  'work',
  'for',
  'the',
  'Olympics',
  '.'],
 ['um',
  'so',
  ',',
  'can',
  'you',
  'tell',
  'me',
  'a',
  'little',
  'bit',
  'about',
  'your',
  'current',
  'job',
  'and',
  'what',
  'like',
  'role',
  'you',
  'play',
  'and',
  'a',
  'little',
  'bit',
  'more',
  'about',
  'the',
  'organization',
  'or',
  'the',
  'company',
  '?'],
 ['gotcha'],
 ['Oh',
  'okay',
  ',',
  'that',
  'makes',


In [23]:
stemmed

[['okay',
  ',',
  'so',
  'i',
  'want',
  'to',
  'begin',
  'with',
  'just',
  'some',
  'basic',
  'demograph',
  'inform',
  'background',
  '.',
  'can',
  'you',
  'just',
  'give',
  'me',
  'a',
  'littl',
  'summari',
  'of',
  'who',
  'you',
  'are',
  'and',
  'your',
  'educ',
  'and',
  'employ',
  'histori',
  '?'],
 ['wow', 'yeah', '.', 'wait', ',', 'were', 'you', 'in', 'the', 'olymp', '?'],
 ['awesom',
  'yeah',
  'that',
  "'",
  's',
  'realli',
  ',',
  'that',
  "'",
  's',
  'a',
  'good',
  'idea',
  'to',
  'work',
  'for',
  'the',
  'olymp',
  '.'],
 ['um',
  'so',
  ',',
  'can',
  'you',
  'tell',
  'me',
  'a',
  'littl',
  'bit',
  'about',
  'your',
  'current',
  'job',
  'and',
  'what',
  'like',
  'role',
  'you',
  'play',
  'and',
  'a',
  'littl',
  'bit',
  'more',
  'about',
  'the',
  'organ',
  'or',
  'the',
  'compani',
  '?'],
 ['gotcha'],
 ['oh',
  'okay',
  ',',
  'that',
  'make',
  'sens',
  ',',
  'but',
  'still',
  'within',
  'maci

In [26]:
flattened_stemmed = []
for sublist in stemmed:
    for word in sublist:
        flattened_stemmed.append(word)

distinct_stemmed = set(flattened_stemmed)

distinct_stemmed

{"'",
 ',',
 '-',
 '.',
 '...',
 '/',
 '1',
 '10',
 ':',
 '?',
 '[',
 ']',
 'a',
 'abl',
 'about',
 'activ',
 'actual',
 'adopt',
 'age',
 'all',
 'alreadi',
 'also',
 'amaz',
 'an',
 'and',
 'ani',
 'anyth',
 'are',
 'as',
 'asian',
 'at',
 'awesom',
 'back',
 'background',
 'base',
 'basi',
 'basic',
 'be',
 'becaus',
 'begin',
 'best',
 'big',
 'bit',
 'born',
 'both',
 'brought',
 'but',
 'by',
 'can',
 'center',
 'certain',
 'chang',
 'common',
 'compani',
 'competit',
 'complain',
 'concern',
 'constrain',
 'context',
 'could',
 'coupl',
 'cultur',
 'current',
 'day',
 'dc',
 'demograph',
 'did',
 'differ',
 'difficult',
 'direct',
 'discuss',
 'distribut',
 'divers',
 'do',
 'don',
 'down',
 'educ',
 'effect',
 'els',
 'email',
 'emot',
 'employ',
 'engin',
 'environ',
 'etcetera',
 'ethnic',
 'ever',
 'everi',
 'exampl',
 'excit',
 'experi',
 'experienc',
 'express',
 'feel',
 'felt',
 'few',
 'fight',
 'first',
 'flyer',
 'follow',
 'for',
 'friendli',
 'from',
 'frustrat',
 '